#Introduction to (why I love) Julia: Part 4 - Metaprogramming
Metaprogamming is **writing code that writes code!**   

Now how cool is that? Pretty cool I know. So I myself again haven't used this feature that much so again the following examples are going to be _borrowed_ from other sources. But these are incredibly cool.   

Used correctly and metaprogramming in Julia can be used to enhance the performance of your code.

_I would like to reiterate at this point that this document is a list of reasons why I like Julia as opposed to a tutorial. For more comprehensive information, go to the official [Julia Documentation](http://docs.julialang.org/en/release-0.3/)_

###Macros
You can think of julia execution as a two-step process. It first parses the code (parse time) and then run the code (run time).   

The metaprogramming in Julia is performed using macros. These are essentialy **functions evaluated at parse-time**, which take a **symbolic expression** as input and produce **another expression** as output, which is **inserted into the code** before compilation (i.e. code generation during parse time):

## parse → expressions → macro → new expr. → compile 

To illustrate, consider this example

In [23]:
macro twostep(arg)
    println("I execute at parse time. The argument is: ", arg)
    
    return :(println("I execute at runtime. The argument is: ", $arg))
end

ex = macroexpand(:(@twostep :("Jonny")))

I execute at parse time. The argument is: :("Jonny")


:(println("I execute at runtime. The argument is: ",:("Jonny")))

The `macroexpand` function allows us the view the quoted return expression. We can see the return expression as the second line of the output above. But notice that the println statement above was **actually executed**. We know that this wasn't executed at run time because the argument wasn't evaluate to the `String` object "Jonny". 

###Another simple example
This macro **reverses the order of function arguments**

In [3]:
macro reverse(ex)
    if isa(ex, Expr) && ex.head == :call
        return Expr(:call, ex.args[1], reverse(ex.args[2:end])...)
    else
        return ex
    end
end

In [5]:
# equivalent to 4 - 1
@reverse 6 - 4

-2

In [7]:
@reverse "backwards?" * "this " * "is "

"is this backwards?"

###A more useful example
Here is an example from a (very useful) Julia Package called **Devectorize.jl**.

In many programming languages (including Julia), expressions are immediately evaluated upon construction. This simple strategy often results in less than optimal behaviors, which, for example, include *creation of unnecessary temporaries* and *repeated memory round-trips*. Consider the following example,

```julia
r = a .* b + c .* d + a
```

With immediate evaluation, three temporaries, respectively for storing the results of ``a .* b``, ``c .* d ``, and ``a .* b + c .* d``. Also, the array ``a`` will be traversed twice. Moreover, computation on large arrays is often memory-bound -- the run-time performance largely depends on how many times you have to scan the arrays. 

For the formula above, a much more efficient way to evaluate it can be expressed using for-loops as follows

```julia
n = length(a)
r = zeros(n)
for i = 1 : n
	r[i] = a[i] * b[i] + c[i] * d[i] + a[i]
end
```

With this piece of code, you can get all the results in one pass, without creating any temporary arrays.
However, low-level for-loops are often much longer and more difficult to read, write, and maintain. 

> **Is it possible to combine the elegance of high-level expressions and the performance of low-level for-loops?**

Yes it is!

We can write something like 
```julia
@devec r = a .* b + c .* d + a
``` 
To obtain the required performance benefits

Here is a table of benchmark results on some typical cases.

|                 |  julia vec |  @devec  | hand-coded loop |
| -------------   | -----------|---------|-----------------|
| simple-ewise    |   1.0000   | 2.6032x |  2.5719x |
| complex-ewise   |   1.0000   | 2.4581x |  2.4364x |
| shift-dot       |   1.0000   | 8.3237x |  8.2959x |
| colwise-sum     |   1.0000   | 1.3321x |  1.2771x |
| rowwise-sum     |   1.0000   | 4.2736x |  4.2444x |
| colwise-eucdist |   1.0000   | 5.6502x |  5.5356x |

*The result was obtained with Julia ``commit 3f92b13210 (2013-02-03)`` on Mac OS X 10.8, using the script ``test/bench_devec.jl``, which comes with the Devectorize package.*